In [1]:
# change the current dir so the sys know the .py modules
import os
os.chdir('/home/jupyter/eye-tracking/python')

In [2]:
import ijson
import base64
import cv2
import random
import requests
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from cnn_utils import *
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

## Get data from the DB

In [3]:
data = requests.get("https://gb.cs.unc.edu/json/drop",headers= { "Accept": "application/json" })

In [4]:
available_ids = []
for drop in data.json()['drops']:
    available_ids.append(drop['id'])
# available_ids

We need to exclude the bad IDs. Here we hardcode the bad IDs by using our [image filter website](http://patrickma.me/simple_filter/) 

In [9]:
bad_ids = [87,88,123,124,125,126,127,128,129,132,133,134,135,136,137,138,148,149,150,151,152,153,156,157,158,159,160,161,166,167,168,169,171,172,173,174,175,176,177,178,179,180,181,182,185,186,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,251,252,253,254,255,256,257,258,259,260,261,262,264,265,266,267,268,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,467,468,488,489,491,492,500,501,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551]

In [13]:
valid_ids = [x for x in available_ids if x not in bad_ids]

In [14]:
allData = []
for valid_id in valid_ids:
    data = requests.get('https://gb.cs.unc.edu/json/drop/'+str(valid_id))
    allData.append(data.json())

In [17]:
len(allData)

606

Save the valid data to a json file

In [18]:
import json
with open('../raw_data/dataset_062920.json', 'w', encoding='utf-8') as f:
    json.dump(allData, f, ensure_ascii=False, indent=4)


In [3]:
import inspect as i
import sys
sys.stdout.write(i.getsource(create_dataframe))


def create_dataframe(relative_dir ='/../raw_data/dataset_062120.json' ):
    """
    convert the JSON file of my research into a dataframe with the following columns:
    eyeImage: np array with shape (25, 50, 3)
    leftEye: np array with shape (6,2): 6 left eye landmarks positions in the form of (x,y), where -1<=x,y<=1
    rightEye: np array with shape (6,2): 6 right eye landmarks positions in the form of (x,y), where -1<=x,y<=1
    y: the position that the user looks at on the screen in the form of (x,y), where -1<=x,y<=1
    """
    f = open(os.path.join(os.path.dirname(__file__))+relative_dir)
    eyeImages,leftEyes,rightEyes,ys = [], [], [], []
    for item in ijson.items(f, "item"):
        # convert the string into a python dict
        temp = json.loads(item) 
        # convert the image into the np array
        eyeImage = convert_base64_to_nparray(temp["x"]["eyeImage"])
        eyeImages.append(eyeImage)

        leftEye = np.array(temp["x"]["eyePositions"]["leftEye"]).resha

In [5]:
df = create_dataframe('/../raw_data/dataset_062920.json')
df.head()

TypeError: the JSON object must be str, bytes or bytearray, not dict

In [ ]:
x_train, x_validation, y_train, y_validation = create_train_validation(df)

In [ ]:
# generate binary y labels
y_train_binary = create_binary_labels(y_train)
y_validation_binary = create_binary_labels(y_validation)

In [ ]:
plot_eyeImages(x_train,y_train_binary)

## Simple CNN

In [ ]:
IMG_WIDTH = x_train["eyeImage"].iloc[0].shape[1]
IMG_HEIGHT = x_train["eyeImage"].iloc[0].shape[0]


In [ ]:
# def forward_propagation(eyeImage, leftEye, rightEye):
#     """
#     Implements the forward propagation for the model:
#     CONV2D -> RELU -> MAXPOOL -> CONV2D -> RELU -> MAXPOOL -> FLATTEN -> FULLYCONNECTED
    
#     Note that for simplicity and grading purposes, we'll hard-code some values
#     such as the stride and kernel (filter) sizes. 
#     Normally, functions should take these values as function parameters.
    
#     Arguments:

#     Returns:
#     Z3 -- the output of the last LINEAR unit
#     """
# #     eyeImage = tf.keras.layers.Input(shape=(IMG_HEIGHT,IMG_WIDTH, 3))
# #     leftEye = tf.keras.layers.Input(shape=(12)) # requires the eye positions to be flattened to 1D
# #     rightEye = tf.keras.layers.Input(shape=(12))
#     conv = tf.keras.layers.Conv2D(filters=20, kernel_size=5, strides=1, activation='relu', kernel_initializer='VarianceScaling')(eyeImage)
#     maxpool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(2, 2))(conv)
#     flat = tf.keras.layers.Flatten()(maxpool)
#     drop = tf.keras.layers.Dropout(rate=0.2)(flat)
#     concat = tf.keras.layers.concatenate([drop, leftEye, rightEye])
#     out = tf.keras.layers.Dense(units=2,activation="tanh",kernel_initializer='VarianceScaling')(concat)

#     simple_model = tf.keras.Model(inputs=[eyeImage, leftEye, rightEye], outputs=out, name="SimpleModel")
#     simple_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),loss = tf.keras.losses.MeanSquaredError())
    
#     return simple_model

### Compile the simple CNN

In [ ]:
tf.convert_to_tensor(x_train["eyeImage"].iloc[0])

In [ ]:
# target = df.pop('y')

# dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))
data = {'X1':  [[1,2,3], [3,4,5]],
        'X2': [['a','b','c'],['c','d','e']]}
df = pd.DataFrame(data)
np.array(df)[1]

In [ ]:
def create_tf_data(X, Y):
    """
    take in the X and Y and transform each column into np array
    """
    
    eyeImage = np.stack(X['eyeImage'].to_numpy())
    leftEye = np.stack(X['leftEye'].to_numpy())
    rightEye = np.stack(X['rightEye'].to_numpy())
    y = np.stack(Y['y'].to_numpy())
    return eyeImage, leftEye, rightEye, y

In [ ]:
eyeImage_train, leftEye_train, rightEye_train, binary_train = create_tf_data(x_train, y_train_binary)
eyeImage_test, leftEye_test, rightEye_test, binary_test = create_tf_data(x_validation, y_validation_binary)

In [ ]:
# encode the y label to make it suitable for CNN
binary_encoder = LabelEncoder()
binary_encoder.fit(binary_train)
binary_train = binary_encoder.transform(binary_train)
binary_test = binary_encoder.transform(binary_test)

In [ ]:
eyeImage = tf.keras.layers.Input(shape=(IMG_HEIGHT,IMG_WIDTH, 3))
leftEye = tf.keras.layers.Input(shape=(12)) # requires the eye positions to be flattened to 1D
rightEye = tf.keras.layers.Input(shape=(12))

conv = tf.keras.layers.Conv2D(filters=20, kernel_size=5, strides=1, activation='relu', kernel_initializer='VarianceScaling')(eyeImage)
maxpool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(2, 2))(conv)
flat = tf.keras.layers.Flatten()(maxpool)
drop = tf.keras.layers.Dropout(rate=0.2)(flat)
concat = tf.keras.layers.concatenate([drop, leftEye, rightEye])
dense = tf.keras.layers.Dense(units=128,activation="relu",kernel_initializer='VarianceScaling')(concat)
out = tf.keras.layers.Dense(units=1,activation="sigmoid")(dense)
simple_model = tf.keras.Model(inputs=[eyeImage, leftEye, rightEye], outputs=out, name="SimpleModel")


In [ ]:
simple_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),metrics=['accuracy'],loss = tf.keras.losses.MeanSquaredError())

simple_model.summary()

In [ ]:
simple_model.fit([eyeImage_train, leftEye_train, rightEye_train], binary_train, batch_size=32, epochs=100)

In [ ]:
test_loss, test_acc = simple_model.evaluate([eyeImage_test, leftEye_test, rightEye_test], binary_test)
print('Test accuracy:', test_acc)